In [1]:
from bqplot import *
from ipywidgets import *
import numpy as np


# Frequency planning tool for RFSoC devices

This Jupyter notebook contains the following:
   
    1) RF Analogue to Digital frequency plan chart
    2) RF Digital to Analogue frequency plan chart
    3) Digital Down Conversion Spectrum
    4) Digital Up Conversion Spectrum

These tools will allow the user to create an in depth frequency plan for any RF project

# 1) RFADC Frequency chart

The following chart provides the relevant information required for the RFADC process. 

Move the FS slider to change the sampling frequency to desired level

Move the Signal center slider to select the center of your signal band

Finally move the Signal bandwidth slider to select the width of your received signal

In [2]:
def Plot_RFADC(Fs,Signal_Center,Signal_Bandwidth):
   
    Nyquist = float(Fs / 2)      #Calculate Nyquist rate
    Fin_Lower = Signal_Center-(Signal_Bandwidth/2) #Calculate lower bound of received frequency
    Fin_Higher = Signal_Center+(Signal_Bandwidth/2) #Calculate upper bound of received frequency
    
    if  Nyquist < Fin_Lower:                #If lower bound is greater than the nyquist, calculate alias to display in 1st Ny Z
        Fin_Higher_Alias = (Fin_Higher-Fs)  #Subtract Fs for aliased frequency
        Fin_Lower_Alias = (Fin_Lower-Fs)
        
        if Fin_Higher_Alias <= 0:              # If signal is negative multiply by -1 for mirror in 1st Nyquist
            Fin_Higher_U = Fin_Lower_Alias*-1
            Fin_Lower_U = Fin_Higher_Alias*-1
        
        else:
            
            Fin_Lower_U = Fin_Lower_Alias     # If signal is positive then use aliased frequencies without mirror
            Fin_Higher_U=Fin_Higher_Alias
        
    else:
        Fin_Lower_U = Fin_Lower               # If signal is less than Nyquist then use values directly, ie not aliased
    
        Fin_Higher_U=Fin_Higher
        
    if Fin_Higher_U > Nyquist:
        Fin_Higher_U = Nyquist
        
    if Fin_Lower_U < 0:
        Fin_Lower_U = 0
        
    HDlist = [] #Create a list for storing HDn values from for loop


    for n in range(2,6,1): #For loop ranging from 2-6, calculates HD2 HD3 HD4 HD5

     if (Fin_Lower_U*n)-Fs + Signal_Bandwidth < -1*Nyquist: #K=0 when equation is less Nyquist *-1 as rhs = negative
         K = 0

 
     elif (Fin_Lower_U*n)-Fs + Signal_Bandwidth < Nyquist: 
        K=1
     
     elif (Fin_Lower_U*n)-Fs + Signal_Bandwidth> Nyquist: 
            K=2
    
     HDn_Lower = (Fin_Lower_U * n) - (Fs * K);    #Calculate lower HD bound
     HDn_Higher = (Fin_Higher_U * n) - (Fs * K);  #Calculate higher HD bound
 

     if HDn_Higher <= 0:        #IF HDn is negative, mirror result 

                 Mirror_HDn_Lower = HDn_Lower * -1;
                 Mirror_HDn_Higher = HDn_Higher * -1;

           #  print("mirrorRange of HD",n, "is", Mirror_HDn_Higher, "to", Mirror_HDn_Lower); # print range
                 temp_list = (Mirror_HDn_Higher, Mirror_HDn_Lower) 
                 HDlist.append(temp_list)        #Update list with calculated values                 
     else:
            # print("Range of HD",n, " is", HDn_Lower, "to", HDn_Higher); #Print range
                 temp_list = (HDn_Lower, HDn_Higher)
                 HDlist.append(temp_list)        #Update list with calculated values
                    
                    
    fsd2m2= [(Fs/2)-HDlist[0][1], (Fs/2)-HDlist[0][0]]    #Calculate Fs/2 - HD2
#print(fsd2m2)

    fsd2m3= [(Fs/2)-HDlist[1][1], (Fs/2)-HDlist[1][0]]    #Calculate Fs/2 - HD3
#print(fsd2m3)

    fsd4m3= [((Fs/4)-HDlist[1][0]), ((Fs/4)-HDlist[1][1])]    #Calculate Fs/4 - HD3   
#print("check",fsd4m3)

    fsd4p3= [(Fs/4)+HDlist[1][0], (Fs/4)+HDlist[1][1]]    #Calculate Fs/4 + HD3
#print("check",fsd4p3)

    fsd8p2= [(Fs/8)+HDlist[0][0], (Fs/8)+HDlist[0][1]]    #Calculate Fs/8 + HD2
#print(fsd8p2)

    fsd8m2= [(Fs/8)+HDlist[0][0], (Fs/8)+HDlist[0][1]]

    fsd8p3= [(Fs/8)+HDlist[1][0], (Fs/8)+HDlist[1][1]]    #Calculate Fs/8 + HD3
#print(fsd8p3)

    fsd8m3= [(Fs/8)-HDlist[1][0], (Fs/8)-HDlist[1][1]]

    fsd4m2= [(Fs/4)-HDlist[0][1], (Fs/4)-HDlist[0][0]]   #Results in error
#print(fsd4m2)

    fsd4p2= [(Fs/4)+HDlist[0][0], (Fs/4)+HDlist[0][1]]   #Results in error
#print(fsd4p2)


    if fsd2m2[1] <= 0:                       #Mirror interleaved spurs if they are negative
        fsd2m2=[fsd2m2[0]*-1,fsd2m2[1]*-1]
    
    if fsd2m3[1] <= 0:
        fsd2m3=[fsd2m3[0]*-1,fsd2m3[1]*-1]
    
    if fsd4m2[1] <= 0:
        fsd4m2=[fsd4m2[0]*-1,fsd4m2[1]*-1]
    
    if fsd4p2[1] <= 0:
        fsd4p2=[fsd4p2[0]*-1,fsd4p2[1]*-1]
    
    if fsd4m3[1] <= 0:
        fsd4m3=[fsd4m3[0]*-1,fsd4m3[1]*-1]

    if fsd4p3[1] <= 0:
        fsd4m3=[fsd4p3[0]*-1,fsd4p3[1]*-1]
 
    if fsd8m2[1] <= 0:
        fsd8m2=[fsd8m2[0]*-1,fsd8m2[1]*-1]

    if fsd8p2[1] <= 0: 
        fsd8p2=[fsd8p2[0]*-1,fsd8p2[1]*-1]

    if fsd8m3[1] <= 0:
        fsd8m3=[fsd8m3[0]*-1,fsd8m3[1]*-1]

    if fsd8p3[1] <= 0:
        fsd8p3=[fsd8p3[0]*-1,fsd8p3[1]*-1]


#If lower bound is negative set = 0

    if fsd2m2[0]<0:
        fsd2m2[0]=0
    if fsd2m3[0]<0:
        fsd2m3[0]=0
    if fsd4m2[0]<0:
        fsd4m2[0]=0
    if fsd4p2[0]<0:
        fsd4p2[0]=0
    if fsd4m3[0]<0:
        fsd4m3[0]=0
    if fsd4p3[0]<0:
        fsd4p3[0]=0
    if fsd8p2[0]<0:
        fsd8p2[0]=0
    if fsd8m2[0]<0:
        fsd8m2[0]=0
    if fsd8p3[0]<0:
        fsd8p3[0]=0
    if fsd8m3[0]<0:
        fsd8m3[0]=0
        
    if fsd2m2[1]>Nyquist:
        fsd2m2[1]=Nyquist
    if fsd2m3[1]>Nyquist:
        fsd2m3[1]=Nyquist
    if fsd4m2[1]>Nyquist:
        fsd4m2[1]=Nyquist
    if fsd4p2[1]>Nyquist:
        fsd4p2[1]=Nyquist
    if fsd4m3[1]>Nyquist:
        fsd4m3[1]=Nyquist
    if fsd4p3[1]>Nyquist:
        fsd4p3[1]=Nyquist
    if fsd8p2[1]>Nyquist:
        fsd8p2[1]=Nyquist
    if fsd8m2[1]>Nyquist:
        fsd8m2[1]=Nyquist
    if fsd8p3[1]>Nyquist:
        fsd8p3[1]=Nyquist
    if fsd8m3[1]>Nyquist:
        fsd8m3[1]=Nyquist
    
    x=fsd2m2[0]        
    y=fsd2m3[0]
    z=fsd4m2[0]

    #Create array for plotting later
    Interleaved_spur_array=[fsd2m2,fsd2m3,fsd4m2,fsd4p2,fsd4m3,fsd4p3,fsd8p2,fsd8m2,fsd8p3,fsd8m3]
    
    #print(Interleaved_spur_array)
                    
    #This section of code sets coordinates used to graph the results
    

        
    Fin = [Fin_Lower_U, Fin_Higher_U]
    #print(Fin_Lower_U)
    #print(Fin_Higher_U)
    #print(Fin)
    Nyquist1 = [Nyquist,Nyquist]
    
    NyHD2 = HDlist[0][1]
    NyHD3 = HDlist[1][1]
    NyHD4 = HDlist[2][1]
    NyHD5 = HDlist[3][1] 
    
    Zero_HD2 = HDlist[0][0]
    Zero_HD3 = HDlist[1][0]
    Zero_HD4 = HDlist[2][0]
    Zero_HD5 = HDlist[3][0]
    
    #if Zero_HD2 < 0:
     #   Zero_HD2 = 0
    #if Zero_HD3 < 0:
     #   Zero_HD3 = 0
    #if Zero_HD4 < 0:
     #   Zero_HD4 = 0
    #if Zero_HD5 < 0:
     #   Zero_HD5 = 0
        
    if NyHD2 > Nyquist:
        NyHD2=Nyquist
    if NyHD3 > Nyquist:
        NyHD3=Nyquist
    if NyHD4 > Nyquist:
        NyHD4=Nyquist
    if NyHD5 > Nyquist:
        NyHD5=Nyquist

    HD2 = [Zero_HD2, NyHD2]    #This accesses the values stored in the list
    HD3 = [Zero_HD3, NyHD3]
    HD4 = [Zero_HD4, NyHD4]
    HD5 = [Zero_HD5, NyHD5]

    HD2_lower = str(HDlist[0][0])
    HD2_Higher = str(HDlist[0][1])

    #y-coordinates
    n1 = [5]
    n2 = [2,2]
    n25 = [2.5, 2.5]
    n3 = [3,3]
    n35 = [3.5,3.5]
    n4 = [4,4]
    n5 = [5,5]
    nn = [5,0]
    
    #Set x and y data used for plotting
    x_data = [HD2, HD3, HD4, HD5]
    y_data = [n2, n3, n4, n5]
    y_inter_spurs =[n25,n35,n25,n25,n35,n35,n25,n25,n35,n35]
    
    
    x_sc = LinearScale()
    y_sc = LinearScale()
    ax_x = Axis(label='Frequency(MHz)', scale=x_sc, tick_format='0.0f')
    ax_y = Axis(label='Harmonic numbers', scale=y_sc,
               orientation='vertical', tick_format='0.2f')
    
    #Create lines for plotting
    spurs = Lines(x=x_data,y=y_data,scales={'x': x_sc, 'y': y_sc}, colors=['red','cyan','green','yellow','pink'],enable_move=False,
                   hovered_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
                   unhovered_style={'opacity': 0.5})
    Fin_horz_plot = Lines(x=Fin,y=n5,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert1_plot = Lines(x=[Fin_Lower_U,Fin_Lower_U],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert2_plot = Lines(x=[Fin_Higher_U,Fin_Higher_U],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Nyquist_plot = Lines(x=Nyquist1,y=nn,scales={'x': x_sc, 'y': y_sc},colors=['red'])
    interleaved_spurs = Lines(x=Interleaved_spur_array,y=y_inter_spurs,scales={'x': x_sc, 'y': y_sc})
    
    tt = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['Frequency', 'harmonic number'])
    tt2 = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['Frequency', 'harmonic number'])
    tt3 = Tooltip(fields=['x'], formats=[''], labels=['Signal Frequency'])
    Nyquisttt = Tooltip(fields=['x'], formats=[''], labels=['Nyquist Frequency'])
    
    spurs.marker='square'
    interleaved_spurs.marker='square'
    Fin_horz_plot.marker='square'
    Nyquist_plot.marker='square'
    #Plot
    fig = Figure(title='RFADC frequency plan',axes=[ax_x, ax_y], marks=[Fin_horz_plot,Fin_vert1_plot, Fin_vert2_plot,spurs,Nyquist_plot])
    spurs.tooltip=tt
    interleaved_spurs.tooltip=tt2
    Fin_horz_plot.tooltip=tt3
    Nyquist_plot.tooltip=Nyquisttt
    tools=Toolbar(figure=fig)

    display(fig,tools)

In [3]:
Fs_slider = widgets.FloatSlider(value=3932.16,min=1900,max=4000,step=100)
Signal_Center_slider = widgets.FloatSlider(value=3932.16,min=100,max=4000,step=50)
Signal_Bandwidth_slider = widgets.FloatSlider(value=100,min=50,max=200,step=50)
widgets.interact(Plot_RFADC,Fs=Fs_slider,Signal_Center=Signal_Center_slider,Signal_Bandwidth=Signal_Bandwidth_slider)

interactive(children=(FloatSlider(value=3932.16, description='Fs', max=4000.0, min=1900.0, step=100.0), FloatS…

<function __main__.Plot_RFADC(Fs, Signal_Center, Signal_Bandwidth)>

# 2) RFDAC frequency plan

The following chart provides the relevant information required for the RFDAC process.

Move the FS slider to change the sampling frequency to desired level

Move the Signal center slider to select the center of your signal band

Finally move the Signal bandwidth slider to select the width of your received signal

In [4]:
def Plot_RFDAC(Fs,Signal_Center,Signal_Bandwidth):
   
    Nyquist = float(Fs / 2)      #Calculate Nyquist rate
    Fin_Lower = Signal_Center-(Signal_Bandwidth/2) #Calculate lower bound of received frequency
    Fin_Higher = Signal_Center+(Signal_Bandwidth/2) #Calculate upper bound of received frequency
    
    if Signal_Center == Nyquist:
        Fin_image_low = 0 - Signal_Bandwidth/2
        Fin_image_high = 0 + Signal_Bandwidth/2
        
    if  Nyquist < Fin_Lower:                #If lower bound is greater than the nyquist, calculate alias to display in 1st Ny Z
        Fin_Higher_Alias = (Fin_Higher-Fs)  #Subtract Fs for aliased frequency
        Fin_Lower_Alias = (Fin_Lower-Fs)
        
        if Fin_Higher_Alias <= 0:              # If signal is negative multiply by -1 for mirror in 1st Nyquist
            Fin_Higher_U = Fin_Lower_Alias*-1
            Fin_Lower_U = Fin_Higher_Alias*-1
        
        else:
            
            Fin_Lower_U = Fin_Lower_Alias     # If signal is positive then use aliased frequencies without mirror
            Fin_Higher_U=Fin_Higher_Alias
        
    else:
        Fin_Lower_U = Fin_Lower               # If signal is less than Nyquist then use values directly, ie not aliased
    
        Fin_Higher_U=Fin_Higher
        
            
    if Fin_Higher_U > Nyquist:
        Fin_Higher_U = Nyquist
        
    if Fin_Lower_U < 0:
        Fin_Lower_U = 0
        
        
    Fin_image_low = Fin_Lower - Fs
    if Fin_image_low < 0:
        Fin_image_low = Fin_image_low*-1
        
    Fin_image_high = Fin_Higher - Fs
    if Fin_image_high < 0:
        Fin_image_high = Fin_image_high* -1
        

    HDlist = [] #Create a list for storing HDn values from for loop


    for n in range(2,6,1): #For loop ranging from 2-6, calculates HD2 HD3 HD4 HD5

     if (Fin_Lower_U*n)-Fs + Signal_Bandwidth < -1*Nyquist: #K=0 when equation is less Nyquist *-1 as rhs = negative
         K = 0

 
     elif (Fin_Lower_U*n)-Fs + Signal_Bandwidth < Nyquist: 
        K=1
     
     elif (Fin_Lower_U*n)-Fs + Signal_Bandwidth> Nyquist: 
            K=2
    
     HDn_Lower = (Fin_Lower_U * n) - (Fs * K);    #Calculate lower HD bound
     HDn_Higher = (Fin_Higher_U * n) - (Fs * K);  #Calculate higher HD bound
 

     if HDn_Higher <= 0:        #IF HDn is negative, mirror result 

                 Mirror_HDn_Lower = HDn_Lower * -1;
                 Mirror_HDn_Higher = HDn_Higher * -1;

                 

           #  print("mirrorRange of HD",n, "is", Mirror_HDn_Higher, "to", Mirror_HDn_Lower); # print range
                 temp_list = (Mirror_HDn_Higher, Mirror_HDn_Lower) 
                 HDlist.append(temp_list)        #Update list with calculated values                 
     else:
            # print("Range of HD",n, " is", HDn_Lower, "to", HDn_Higher); #Print range
                 temp_list = (HDn_Lower, HDn_Higher)
                 HDlist.append(temp_list)        #Update list with calculated values
   


    HDlist2 = [] #Create a list for storing HDn Nyquist 2 values from for loop


    for n in range(2,6,1): #For loop ranging from 2-6, calculates HD2 HD3 HD4 HD5 Nyquist 2

     if (Fin_Lower_U*n)-Fs + Signal_Bandwidth < -1*Nyquist: #K=0 when equation is less Nyquist *-1 as rhs = negative
         K = 1

 
     if (Fin_Lower_U*n)-Fs + Signal_Bandwidth < Nyquist: 
        K=2
            
    
     if (Fin_Lower_U*n)-Fs + Signal_Bandwidth > Nyquist:
            K=3
       
    
     if (Fin_Lower_U*n)-Fs - Signal_Bandwidth <= -1*Nyquist:
            K=1
    
     test = (Fin_Lower_U*n)-Fs+Signal_Bandwidth
     print(test)
     print(K)
     HDn_Lower = (Fin_Lower_U * n) - (Fs * K);    #Calculate lower HD bound
     HDn_Higher = (Fin_Higher_U * n) - (Fs * K);  #Calculate higher HD bound
 

     if HDn_Higher <= 0:        #IF HDn is negative, mirror result 

                 Mirror_HDn_Lower = HDn_Lower * -1;
                 Mirror_HDn_Higher = HDn_Higher * -1;

                 

           #  print("mirrorRange of HD",n, "is", Mirror_HDn_Higher, "to", Mirror_HDn_Lower); # print range
                 temp_list2 = (Mirror_HDn_Higher, Mirror_HDn_Lower) 
                 HDlist2.append(temp_list2)        #Update list with calculated values                 
     else:
            # print("Range of HD",n, " is", HDn_Lower, "to", HDn_Higher); #Print range
                 temp_list2 = (HDn_Lower, HDn_Higher)
                 HDlist2.append(temp_list2)        #Update list with calculated values

                    
    #This section of code sets coordinates used to graph the result


    Fin_image=[Fin_image_low, Fin_image_high]    
    Fin_original = [Fin_Lower, Fin_Higher]
    print(Fin_Lower_U)
    print(Fin_Higher_U)
    print("image",Fin_image)
    print(Fin_Lower)
    print(Fin_Higher)
    print(Fin_original)
    Nyquist1 = [Nyquist,Nyquist]
    
    NyHD2 = HDlist[0][1]
    NyHD3 = HDlist[1][1]
    NyHD4 = HDlist[2][1]
    NyHD5 = HDlist[3][1] 
    
    
    if NyHD2 > Nyquist:
        NyHD2=Nyquist
    if NyHD3 > Nyquist:
        NyHD3=Nyquist
    if NyHD4 > Nyquist:
        NyHD4=Nyquist
    if NyHD5 > Nyquist:
        NyHD5=Nyquist

    HD2 = [HDlist[0][0], NyHD2]    #This accesses the values stored in the list
    HD3 = [HDlist[1][0], NyHD3]
    HD4 = [HDlist[2][0], NyHD4]
    HD5 = [HDlist[3][0], NyHD5]
    
    NyHD2_2 = HDlist2[0][0]
    NyHD3_2 = HDlist2[1][0]
    NyHD4_2 = HDlist2[2][0]
    NyHD5_2 = HDlist2[3][0] 
    
    FsHD2_2 = HDlist2[0][1]
    FsHD2_3 = HDlist2[1][1]
    FsHD2_4 = HDlist2[2][1]
    FsHD2_5 = HDlist2[3][1]
    
    print ("hd2 one",FsHD2_2)
    print("Hd2 two", NyHD2_2)
    
    if NyHD2_2 < Nyquist:
        NyHD2_2=Nyquist
    if NyHD3_2 < Nyquist:
        NyHD3_2=Nyquist
    if NyHD4_2 < Nyquist:
        NyHD4_2=Nyquist
    if NyHD5_2 < Nyquist:
        NyHD5_2=Nyquist
        
    if NyHD2_2 == Fs:
        Fv= FsHD2_2 - Fs
        FsHD2_2 = Fs-Fv
    if NyHD3_2 == Fs:
        Ft= FsHD3_2 - Fs
        FsHD2_3 = Fs-Ft
    if NyHD4_2 == Fs:
        Fu= FsHD2_4 - Fs
        FsHD2_4 = Fs-Fu
    if NyHD5_2 == Fs:
        Fp= FsHD5_2 - Fs
        FsHD2_5 = Fs-Fp
    

    HD22 = [NyHD2_2,FsHD2_2]    #This accesses the values stored in the list
    HD32 = [NyHD3_2,FsHD2_3]
    HD42 = [NyHD4_2,FsHD2_4]
    HD52 = [NyHD5_2,FsHD2_5]
    
    
    print(HD22)
    print(HD42)
    print(HD52)

    HD2_lower = str(HDlist[0][0])
    HD2_Higher = str(HDlist[0][1])
    
    print("This is",NyHD5_2)

    #y-coordinates
    n1 = [5]
    n2 = [2,2]
    n3 = [3,3]
    n4 = [4,4]
    n5 = [5,5]
    nn = [5,0]
    
    #Set x and y data used for plotting
    x_data = [HD2, HD3, HD4, HD5]
    x_data2 = [HD22, HD32, HD42, HD52]
    y_data = [n2, n3, n4, n5]

    
    
    x_sc = LinearScale()
    y_sc = LinearScale()
    ax_x = Axis(label='Frequency(MHz)', scale=x_sc, tick_format='0.0f')
    ax_y = Axis(label='Harmonic numbers', scale=y_sc,
               orientation='vertical', tick_format='0.2f')
    
    #Create lines for plotting
    spurs = Lines(x=x_data,y=y_data,scales={'x': x_sc, 'y': y_sc}, colors=['red','cyan','green','yellow','pink'],enable_move=False,
                   hovered_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
                   unhovered_style={'opacity': 0.5})
    spurs2 = Lines(x=x_data2,y=y_data,scales={'x': x_sc, 'y': y_sc}, colors=['red','cyan','green','yellow','pink'],enable_move=False,
                   hovered_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
                   unhovered_style={'opacity': 0.5})
    Fin_horz_plot = Lines(x=Fin_image,y=n5,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert1_plot = Lines(x=[Fin_image_low,Fin_image_low],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert2_plot = Lines(x=[Fin_image_high,Fin_image_high],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_horz_plot_original = Lines(x=Fin_original,y=n5,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert1_plot_original = Lines(x=[Fin_Lower,Fin_Lower],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert2_plot_original = Lines(x=[Fin_Higher,Fin_Higher],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Nyquist_plot = Lines(x=Nyquist1,y=nn,scales={'x': x_sc, 'y': y_sc},colors=['red'])
    
    tt = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['Frequency', 'harmonic number'])
    
    tt_fim = Tooltip(fields=['x'], formats=[''], labels=['Signal Frequency image'])
    tt3 = Tooltip(fields=['x'], formats=[''], labels=['Signal Frequency'])
    Nyquisttt = Tooltip(fields=['x'], formats=[''], labels=['Nyquist Frequency'])
    spurs.marker='square'
    spurs2.marker='square'
    Fin_horz_plot.marker='square'
    Fin_horz_plot_original.marker='square'
    Nyquist_plot.marker='square'
    #Plot
    fig = Figure(title='RFDAC frequency plan',axes=[ax_x, ax_y], marks=[Fin_horz_plot_original,Fin_vert1_plot_original,Fin_vert2_plot_original,Fin_horz_plot,Fin_vert1_plot, Fin_vert2_plot,spurs,spurs2,Nyquist_plot])
    spurs.tooltip=tt
    spurs2.tooltip=tt
    Fin_horz_plot.tooltip=tt_fim
    Fin_horz_plot_original.tooltip=tt3
    Nyquist_plot.tooltip=Nyquisttt
    tools=Toolbar(figure=fig)

    display(fig,tools)

In [5]:
#Fs_slider = widgets.FloatSlider(value=3932.16,min=3900,max=4000,step=0.1)
Signal_Center= widgets.FloatSlider(value=100,min=100,max=4000,step=50)
Signal_Bandwidth = widgets.FloatSlider(value=100,min=50,max=200,step=50)
widgets.interact(Plot_RFDAC,Fs=Fs_slider,Signal_Center=Signal_Center,Signal_Bandwidth=Signal_Bandwidth)

interactive(children=(FloatSlider(value=3932.16, description='Fs', max=4000.0, min=1900.0, step=100.0), FloatS…

<function __main__.Plot_RFDAC(Fs, Signal_Center, Signal_Bandwidth)>

#  3) DUC Spectrum

The following spectrum will display the relevevant information for the Digital up conversion process.
If mix mode is on Nyquist zone 2 will show the relevant spurs.
If normal mode is on Nyquist zone 1 will show the relevant spurs. 
The spectrum shows both Nyquist zones so user does not need to make changes depending on which mode they operate.

The Fs slider will select the users desired Sampling Frequency
The Frequency_in slider will select the desired input signal
The NCO slider will select how much the input signal has been shifted by the NCO
The HD2/HD3 noise sliders will select the noise level for the harmonic spurs in dB

In [6]:
def DUC_spur_calculator(Fs, Frequency_in, NCO):
    
    Nyquist = float(Fs / 2)      #Calculate Nyquist rate
    Fundamental = NCO + Frequency_in
    F_image_calc = Fundamental - Fs

    print(F_image_calc)
    if  Nyquist < F_image_calc:
        F_image = F_image_calc-Fs
        F_image = F_image *-1
    
    
    print("The freq in", Frequency_in)
    print("sample", Fs)
    
    Nyquist = Fs/2 #Calculate Nyquist value
    HDlist = [] #Create a list for storing loop values
    
    # Calculate K variable, K increases as conditions are met
    
    for n in range (2,4,1):
       
        print("fincheck",Frequency_in*n)
        
        if Fundamental*n < Nyquist:
            K = 0
        
        elif Fundamental*n <= Fs + Nyquist:
            K = 1
    
        elif Fundamental*n <= 2*Fs + Nyquist:
            K = 2
        
        elif Fundamental*n <= 3*Fs + Nyquist:
            K = 3
        
        elif Fundamental*n <= 4*Fs + Nyquist:
            K = 4
            
        elif Fundamental*n <= 5*Fs + Nyquist:
            K = 5
        else:
            K = 6
        
        print("K check",K)
        HDn = (Fundamental *n) - (Fs*K)
        
        if HDn < 0:
            HDn = HDn*-1
        print("original",HDn)


        if HDn <= 0:        #IF HDn is negative, mirror result 

                    Mirror_HDn = HDn * -1;
                 
           #  print("mirrorRange of HD",n, "is", Mirror_HDn_Higher, "to", Mirror_HDn_Lower); # print range
                    temp_list = (Mirror_HDn) 
                    HDlist.append(temp_list)        #Update list with calculated values                 
        else:
            # print("Range of HD",n, " is", HDn_Lower, "to", HDn_Higher); #Print range
                    temp_list = (HDn)
                    HDlist.append(temp_list)        #Update list with calculated values

    
    HD2_value = HDlist[0]
    HD3_value = HDlist[1]
    
    return [HD2_value, HD3_value]


In [7]:
def Plot_DUC(Fs,Frequency_in,NCO,HD2_noise,HD3_noise):
   
    Nyquist = float(Fs / 2)      #Calculate Nyquist rate
    Fundamental = NCO + Frequency_in
    F_image = Fundamental - Fs

    print(F_image)
    
    #if F_image_calc < 0:
       # F_image = F_image * -1
        
    if  Nyquist < F_image:
        F_image = F_image-Fs
        F_image = F_image *-1
    
    HD = DUC_spur_calculator(Fs,Frequency_in,NCO)

    HD2 = HD[0]
    HD3 = HD[1]
    
    x_plot_HD2 = [HD2, HD2]
    x_plot_HD3 = [HD3, HD3]

    y_plot_HD2 = [-1*HD2_noise,-100]
    y_plot_HD3 = [-1*HD3_noise,-100]
    
    x_data = [x_plot_HD2,x_plot_HD3]
    y_data = [y_plot_HD2,y_plot_HD3]
    fund_x = [Fundamental,Fundamental]
    fim_x = [F_image, F_image]
    Fs_x = [Fs,Fs]
    fund_y = [0,-100]
    Nyquist1 = [Nyquist,Nyquist]
    
    x_sc = LinearScale()
    y_sc = LinearScale()
    ax_x = Axis(label='Frequency(MHz)', scale=x_sc, tick_format='0.0f')
    ax_y = Axis(label='Amplitude(dB)', scale=y_sc,
               orientation='vertical', tick_format='0.2f')
    
    #Create lines for plotting
    spurs = Lines(x=x_data,y=y_data,scales={'x': x_sc, 'y': y_sc}, colors=['cyan','pink'],enable_move=False,
                   hovered_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
                   unhovered_style={'opacity': 0.5},labels=['HD2','HD3'],display_legend=True)
    Fundamental_plot = Lines(x=fund_x,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['blue'],labels=['Fundamental'],display_legend=True)
    Fim_plot = Lines(x=fim_x,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['green'],labels=['F image'],display_legend=True)
    Nyquist_plot = Lines(x=Nyquist1,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['red'],labels=['Nyquist'],display_legend=True)
    Fs_plot = Lines(x=Fs_x,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['orange'],labels=['Fs'],display_legend=True)
   
    tt = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['Frequency', 'dB level'])

    tt3 = Tooltip(fields=['x'], formats=[''], labels=['Frequency'])
    Nyquisttt = Tooltip(fields=['x'], formats=[''], labels=['Nyquist Frequency'])
    
    spurs.marker='square'
    Fundamental_plot.marker='diamond'
    Fim_plot.marker='diamond'
    Fs_plot.marker='cross'
    Nyquist_plot.marker = 'cross'

    
    fig = Figure(title='DUC Spectrum',axes=[ax_x, ax_y], marks=[spurs,Fundamental_plot,Fs_plot,Nyquist_plot,Fim_plot],
                     display_legend=True)

    labels = ['spurs','Fundamental_plot','Fs_plot','Nyquist_plot','Fim_plot']
    
    spurs.tooltip=tt
    Fundamental_plot.tooltip=tt
    Nyquist_plot.tooltip=Nyquisttt
    Fs_plot.tooltip=tt
    Fim_plot.tooltip=tt
    tools=Toolbar(figure=fig)
    
        
    display(fig, tools)

In [8]:
Fs_slider = widgets.FloatSlider(value=3932.16,min=1900,max=4000,step=100)
Signal_Center_slider = widgets.FloatSlider(value=3500,min=100,max=4000,step=50)
NCO_slider = widgets.FloatSlider(value=3500,min=100,max=4000, step = 50)
HD2_slider = widgets.FloatSlider(value=10, min=0, max=200, step = 10)
HD3_slider = widgets.FloatSlider(value=10, min=0, max=200, step = 10)
widgets.interact(Plot_DUC,Fs=Fs_slider,Frequency_in=Signal_Center_slider,NCO=NCO_slider,HD2_noise=HD2_slider,HD3_noise=HD3_slider)

interactive(children=(FloatSlider(value=3932.16, description='Fs', max=4000.0, min=1900.0, step=100.0), FloatS…

<function __main__.Plot_DUC(Fs, Frequency_in, NCO, HD2_noise, HD3_noise)>

# DDC Spectrum

The following spectrum displays the relevant information for the Digital down conversion process. Note the values displayed are the decimated values. Any values exceeding the decimated Nquist value are folded back until they are present in the display band.

The Fs slider will select the users desired Sampling Frequency
The Frequency_in slider will select the desired input signal
The NCO slider will select how much the input signal has been shifted by the NCO
The HD2/HD3 noise sliders will select the noise level for the harmonic spurs in dB
The radio button for Decimation selects the decimation factor of the signal
The radio button for int_ADC selects the number of interleaved ADC's the user has

In [9]:
def Equation_calculator(Fs, Frequency_in, NCO, Decimation , Interleaved_ADC):
    
    print("The freq in", Frequency_in)
    print("sample", Fs)
    print("NCO",NCO)
    
    Nyquist = Fs/2 #Calculate Nyquist value
    
    # Calculate K variable, K increases as conditions are met
    
    if Frequency_in < Nyquist:
        K=0
        
    elif Frequency_in > Nyquist:
        K = 1
        
    elif Frequency_in > Fs + Nyquist:
        K = 2
    
    elif Frequency_in > 2*Fs + Nyquist:
        K = 3
        
    elif Frequency_in > 3*Fs + Nyquist:
        K = 4
        
    else:
        K = 5
        
    ## Potentially add error statement if input frequency reaches unrealistic value 
    
    #Calculate input alias before NCO shift
    Input = (Frequency_in - Fs*K)
    
    #Mirror input if value is negative
    if Input < 0:
        Input=Input*-1
        print("input is mirrored",Input)
     
    #Shift input by NCO
    Input_NCO = (Input) - NCO
    
    #Calculate final Answer for input alias 
    Answer = Input_NCO + (Interleaved_ADC * Fs/Decimation)
    
    #Use decimated Nyquist and Fs values for calculating signal in 1st decimated Nyquist zone
    Decimated_Nyquist = Nyquist/Decimation
    Decimated_Fs = Fs/Decimation
    
    print("Decimated Nyquist",Decimated_Nyquist)
    print("Decimated Fs",Decimated_Fs)
    
    #Use following process to continuously subtract decimated FS until value is displayed in first decimated Nyquist zone
    stopprocess = False

    while stopprocess == False:

        if Answer > Decimated_Nyquist:
            Answer = Answer-Decimated_Fs
            
            #Set to true when value is below decimated Nyquist
            if Answer < Decimated_Nyquist:
                stopprocess = True
             
    print("in alias",Answer)
   
    return Answer

In [10]:
def DDC_spur_calculator(Fs, Frequency_in, NCO, Decimation , Interleaved_ADC):
    
    print("The freq in", Frequency_in)
    print("sample", Fs)
    
    Nyquist = Fs/2 #Calculate Nyquist value
    HDlist = [] #Create a list for storing loop values
    
    # Calculate K variable, K increases as conditions are met
    
    for n in range (2,4,1):
       
        print("fincheck",Frequency_in*n)
        
        if Frequency_in*n < Nyquist:
            K = 0
        
        elif Frequency_in*n <= Fs + Nyquist:
            K = 1
    
        elif Frequency_in*n <= 2*Fs + Nyquist:
            K = 2
        
        elif Frequency_in*n <= 3*Fs + Nyquist:
            K = 3
        
        else:
            K = 4
        
        print("K check",K)
        hdn_orig = (Frequency_in*n) - (Fs*K)
        
        if hdn_orig < 0:
            hdn_orig = hdn_orig*-1
        print("original",hdn_orig)
        hd_NCO = hdn_orig - NCO
        print("NCO",hd_NCO)
        #Calculate final Answer for input alias 
        Answer = hd_NCO + (Interleaved_ADC * Fs/Decimation)
        
        print("This is answer",Answer)
        
        Decimated_Nyquist = Nyquist/Decimation
        Decimated_Fs = Fs/Decimation
    
        
        #Use following process to continuously subtract decimated FS until value is displayed in first decimated Nyquist zone
        stopprocess = False

        while stopprocess == False:
            if Answer > Decimated_Nyquist:
                Answer = Answer-Decimated_Fs
            
            #Set to true when value is below decimated Nyquist
                if Answer < Decimated_Nyquist:
                    stoprocess = True
                    temp_list = (Answer)
                    HDlist.append(temp_list) 
                    print("Final_HD",Answer)
                    Answer = 0
                    print("this is answer 2",Answer)
                    break
        

    
    HD2_value = HDlist[0]
    HD3_value = HDlist[1]
    
    return [HD2_value, HD3_value]


In [11]:
def Plot_DDC(Fs, Frequency_in,NCO,HD2_noise,HD3_noise, Decimation, Interleaved_ADC):
    
    Nyquist = Fs/2
    Decimated_Nyquist = Nyquist/Decimation
    print(Decimation)
    print(Interleaved_ADC)
    
    input_alias = Equation_calculator(Fs,Frequency_in,NCO,Decimation,Interleaved_ADC)
    print(input_alias)
    
    HD = DDC_spur_calculator(Fs,Frequency_in,NCO, Decimation, Interleaved_ADC)
    print(HD)
    print("HD2",HD[0])
    print("HD3",HD[1])
    
    ##
    
    y_plot_HD2 = [-1*HD2_noise,-100]
    y_plot_HD3 = [-1*HD3_noise,-100]

    HD2 = HD[0]
    HD3 = HD[1]
    
    x_plot_HD2 = [HD2, HD2]
    x_plot_HD3 = [HD3, HD3]
    
    x_data = [x_plot_HD2,x_plot_HD3]
    y_data = [y_plot_HD2,y_plot_HD3]
    f_in = [input_alias,input_alias]
    #fim_x = [F_image, F_image]
    fund_y = [0,-100]
    Decimated_Nyquist_plot = [Decimated_Nyquist,Decimated_Nyquist]
    
    x_sc = LinearScale()
    y_sc = LinearScale()
    ax_x = Axis(label='Frequency(MHz)', scale=x_sc, tick_format='0.0f')
    ax_y = Axis(label='Amplitude(dB)', scale=y_sc,
               orientation='vertical', tick_format='0.2f')
    
    #Create lines for plotting
    spurs = Lines(x=x_data,y=y_data,scales={'x': x_sc, 'y': y_sc}, colors=['cyan','cyan'],enable_move=False,
                   hovered_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
                   unhovered_style={'opacity': 0.5})
    f_in = Lines(x=f_in,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    #Fim_plot = Lines(x=fim_x,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Decimated_Nyquist_plot = Lines(x=Decimated_Nyquist_plot,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['red'])
    #Fs_plot = Lines(x=Fs_x,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['red'])
   
    tt = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['Frequency', 'dB level'])

    tt3 = Tooltip(fields=['x'], formats=[''], labels=['Frequency'])
    
    spurs.marker='square'
    f_in.marker='diamond'
    #Fim_plot.marker='diamond'
    #Fs_plot.marker='cross'
    Decimated_Nyquist_plot.marker = 'cross'

    
    fig = Figure(title='DDC Spectrum',axes=[ax_x, ax_y], marks=[spurs,f_in,Decimated_Nyquist_plot])
    
    spurs.tooltip=tt
    f_in.tooltip=tt
    Decimated_Nyquist_plot.tooltip=tt
    #Fim_plot.tooltip=tt
    tools=Toolbar(figure=fig)
    
        
    display(fig, tools)

In [12]:
#Create slider widgets to control variables
Fs_slider = widgets.FloatSlider(value=2000,min=1900,max=4000,step=100)
Frequency_in_slider = widgets.FloatSlider(value=500,min=100,max=4000,step=50)
NCO_slider = widgets.FloatSlider(value=100, min=100, max=4000, step = 100)
HD2_slider = widgets.FloatSlider(value=10, min=0, max=200, step = 10)
HD3_slider = widgets.FloatSlider(value=10, min=0, max=200, step = 10)

#Create button widgets to control variables
Decimation_buttons = widgets.RadioButtons(
    options=[2,4,8,12],
    value=12,
    description='Decimation:',
    disabled=False
)

Interleaved_ADC_buttons = widgets.RadioButtons(
    options=[4,8],
    value=8,
    description='Int ADCs:',
    disabled=False
)


In [13]:
#Allow widgets to interact with DDC plot
widgets.interact(Plot_DDC,Fs=Fs_slider,Frequency_in=Frequency_in_slider,NCO=NCO_slider, HD2_noise = HD2_slider, HD3_noise = HD3_slider, Decimation=Decimation_buttons, Interleaved_ADC=Interleaved_ADC_buttons)

interactive(children=(FloatSlider(value=2000.0, description='Fs', max=4000.0, min=1900.0, step=100.0), FloatSl…

<function __main__.Plot_DDC(Fs, Frequency_in, NCO, HD2_noise, HD3_noise, Decimation, Interleaved_ADC)>